In [170]:
import os
import pandas as pd
import h5py
from keras.applications import *
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.utils import multi_gpu_model
from keras.models import load_model
import random
import time
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import pickle as pkl
import matplotlib
rootPathList=['data']
CHECK_DRIVERS_NUMBER = 3
drivers_to_files=pd.read_csv('driver_imgs_list.csv')

In [72]:
def recover_data(rootPathList):
    print('开始重新恢复初始数据集。。。。。。')
    for i,rootPath in enumerate(rootPathList):           
        for  filePath in (os.listdir(rootPath + '/valid')):
            files = os.listdir(rootPath + '/valid/'+filePath)
            i = filePath[-1]
            for file in files:
                targetFile = (rootPath + '/train' + '/c%s/' + file) % (i )
                sourceFile = (rootPath + '/valid' + '/c%s/'+ file) % (i )
                if (not os.path.exists(targetFile)) and os.path.exists(sourceFile):
                    os.rename(sourceFile, targetFile)
    print('数据已经恢复完毕，可以开始重新挑选数据集')

In [73]:
def pick_train_data(num,rootPathList,CHECK_DRIVERS_NUMBER):
    print('开始挑选本次训练的数据集')
    for i,rootPath in enumerate(rootPathList): 
        check_drivers = drivers_to_files.drop_duplicates(['subject'])['subject'].sample(n=CHECK_DRIVERS_NUMBER, random_state=(num+209))
        print('对于第 %d次训练,抽取的验证集司机编号为：'% num)
        print(check_drivers)
        for driver in check_drivers:
            check_drivers_dataFrame = drivers_to_files.loc[drivers_to_files['subject'] == driver]
            for indexs in check_drivers_dataFrame.index:
                filePath = check_drivers_dataFrame.loc[indexs]
                sourceFile = (rootPath + '/train' + '/' + filePath['classname'] + '/' + filePath['img'])
                targetFile = (rootPath + '/valid' + '/' + filePath['classname'] + '/' + filePath['img'])
                if (not os.path.exists(targetFile)) and os.path.exists(sourceFile):
                    os.rename(sourceFile, targetFile)
    print('数据集挑选完毕，可以开始训练过程')

In [74]:
recover_data(rootPathList)
pick_train_data(0,rootPathList,CHECK_DRIVERS_NUMBER)

开始重新恢复初始数据集。。。。。。
数据已经恢复完毕，可以开始重新挑选数据集
开始挑选本次训练的数据集
对于第 0次训练,抽取的验证集司机编号为：
17778    p061
16984    p056
13523    p049
Name: subject, dtype: object
数据集挑选完毕，可以开始训练过程


In [75]:
def ImageDataGenerator_select(funtion_name):
    datagen = None
    imgesize = None
    if funtion_name == 'DenseNet201':
        imgesize = (224,224)
        datagen = ImageDataGenerator(
            preprocessing_function=densenet.preprocess_input)
    if funtion_name == 'InceptionResNetV2':
        imgesize = (299,299)
        datagen = ImageDataGenerator(
            preprocessing_function=inception_resnet_v2.preprocess_input)
    if funtion_name == 'InceptionV3':
        imgesize = (299,299)
        datagen = ImageDataGenerator(
            preprocessing_function=inception_v3.preprocess_input)
    if funtion_name == 'Xception':
        imgesize = (299,299)
        datagen = ImageDataGenerator(
            preprocessing_function=xception.preprocess_input)
    if funtion_name == 'ResNet50':
        imgesize = (224,224)
        datagen = ImageDataGenerator(
            preprocessing_function=resnet50.preprocess_input)
    if funtion_name == 'VGG19':
        imgesize = (224,224)
        datagen = ImageDataGenerator(
            preprocessing_function=vgg19.preprocess_input)
    if funtion_name == 'VGG16':
        imgesize = (224,224)
        datagen = ImageDataGenerator(
            preprocessing_function=vgg16.preprocess_input)
    if funtion_name == 'NASNetLarge':
        imgesize = (331,331)
        datagen = ImageDataGenerator(
            preprocessing_function=nasnet.preprocess_input)
    if funtion_name == 'NASNetMobile':
        imgesize = (224,224)
        datagen = ImageDataGenerator(
            preprocessing_function=nasnet.preprocess_input)
    return datagen,imgesize

In [113]:
extra_class = 0
rootFilePath ='data'
funtion_name = 'VGG16'

datagen,imageSize = ImageDataGenerator_select('VGG16')

train_generator_label_batch_size = 32
train_generator_nolabel_batch_size = 32
valid_generator_batch_size = 32

In [146]:
class DataSet:
    def __init__(self, batch_size, image_size, datagen, steps_per_epoch, valid_steps_per_epoch, alpha=0.5):

        self.label_batch_size = int(batch_size*alpha)
        self.nolabel_batch_size = batch_size - self.label_batch_size

        self.train_generator_label = datagen.flow_from_directory(
            rootFilePath+'/train',
            target_size=image_size,
            batch_size=self.label_batch_size,
            class_mode='sparse')
        self.train_generator_nolabel = datagen.flow_from_directory(
            rootFilePath + '/test1',
            target_size=image_size,
            batch_size=self.nolabel_batch_size,
            class_mode='sparse')
        self.valid_generator = datagen.flow_from_directory(
            rootFilePath + '/valid',
            target_size=image_size,
            batch_size=batch_size,
            class_mode='sparse')

        self.steps_per_epoch = steps_per_epoch // batch_size * 2
        self.valid_steps_per_epoch = self.valid_generator.samples //batch_size
        
    def test(self):
        print (next(self.train_generator_label)[1])
        
    def get_batches(self):
        for i in range(self.steps_per_epoch):
            train_label = next(self.train_generator_label)
            train_nolabel = next(self.train_generator_nolabel)

            train_label_x = train_label[0]
            train_label_y = train_label[1].reshape(-1, 1)

            train_nolabel_x =train_nolabel[0]
            train_nolabel_y = np.zeros((train_nolabel[0].shape[0], 1),dtype = 'int64')

            train_x_batch = np.concatenate((train_label_x, train_nolabel_x))
            train_y_batch = np.concatenate((train_label_y, train_nolabel_y))
            
            train_label_mask = np.ones_like(train_label_y, dtype='int64')
            train_nolabel_mask = np.zeros_like(train_nolabel_y, dtype ='int64')
            label_mask = np.concatenate((train_label_mask, train_nolabel_mask))

            train_x_batch, train_y_batch, label_mask = shuffle(train_x_batch, train_y_batch, label_mask)

            yield train_x_batch, train_y_batch, label_mask
            
    def get_valid_batches(self):       
        for i in range(self.valid_steps_per_epoch):
            valid_batch = self.valid_generator.next()
            valid_x_batch = valid_batch[0]
            valid_y_batch = (valid_batch[1]).reshape(-1, 1)
            yield valid_x_batch, valid_y_batch


In [121]:
def train_batch(num , train_generator_label, train_generator_nolabel,nolabel_batch_size,label_batch_size):
    
    for i in range(num):

        train_label = train_generator_label.next()
        train_nolabel = train_generator_nolabel.next()

        train_nolabel_y = np.zeros((nolabel_batch_size, 1))
        train_nolabel_y.dtype ='int64'
        train_label_y =np.argmax(train_label[1][:label_batch_size],axis = 1).reshape(-1,1)
       

        train_x_batch = np.concatenate((train_label[0], train_nolabel[0]))
        train_y_batch = np.concatenate((train_label_y, train_nolabel_y))
        
        
        train_label_mask = np.ones_like(train_label_y)
        train_nolabel_mask = np.zeros_like(train_nolabel_y)
        
        label_mask = np.concatenate((train_label_mask, train_nolabel_mask))

        train_x_batch, train_y_batch, label_mask = shuffle(train_x_batch, train_y_batch,label_mask)
 
        label_mask.dtype = 'int64'

        yield train_x_batch, train_y_batch, label_mask

In [122]:
def valid_batch(num , valid_generator):

    for i in range(num):
        valid_batch = valid_generator.next()
        valid_x_batch = valid_batch[0]
        valid_y_batch = np.argmax(valid_batch[1],axis = 1).reshape(-1,1)
        yield valid_x_batch, valid_y_batch

In [129]:
real_size = (224,224,3)
z_size = 200
learning_rate = 0.0003
batch_size = 128
imageSize = (224,224)


    
# label_batch_size = batch_size//2
# nobel_batch_size = batch_size - label_batch_size

# train_generator_label = datagen.flow_from_directory(
#             rootFilePath+'/train',  # this is the target directory
#             target_size=imageSize,  # all images will be resized to 150x150
#             batch_size=label_batch_size,
#             class_mode='categorical')

# train_generator_nolabel = datagen.flow_from_directory(
#         rootFilePath+'/test1',  # this is the target directory
#         target_size=imageSize,  # all images will be resized to 150x150
#         batch_size=nobel_batch_size,
#         class_mode='categorical')

# valid_generator = datagen.flow_from_directory(
#         rootFilePath+'/valid',  # this is the target directory
#         target_size=imageSize,  # all images will be resized to 150x150
#         batch_size=batch_size,
#         class_mode='categorical')

# steps_per_epoch = int(train_generator_label.samples/batch_size/2)
# valid_steps_per_epoch = int(valid_generator.samples/batch_size)

In [79]:
def model_inputs(real_dim, z_dim):
    inputs_real = tf.placeholder(tf.float32, (None, *real_dim), name='input_real')
    inputs_z = tf.placeholder(tf.float32, (None, z_dim), name='input_z')
    y = tf.placeholder(tf.int32, (None), name='y')
    label_mask = tf.placeholder(tf.int32, (None), name='label_mask')
    
    return inputs_real, inputs_z, y, label_mask

In [80]:
def generator(z, output_dim, reuse=False, alpha=0.2, training=True, size_mult=128):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        x1 = tf.layers.dense(z, 4 * 4 * size_mult * 4)
        # Reshape it to start the convolutional stack
        x1 = tf.reshape(x1, (-1, 4, 4, size_mult * 4))
        x1 = tf.layers.batch_normalization(x1, training=training)
        x1 = tf.maximum(alpha * x1, x1)
        
        x2 = tf.layers.conv2d_transpose(x1, size_mult * 2, 5, strides=2, padding='same')
        x2 = tf.layers.batch_normalization(x2, training=training)
        x2 = tf.maximum(alpha * x2, x2)
        
        x3 = tf.layers.conv2d_transpose(x2, size_mult, 5, strides=2, padding='same')
        x3 = tf.layers.batch_normalization(x3, training=training)
        x3 = tf.maximum(alpha * x3, x3)
        
        # Output layer
        logits = tf.layers.conv2d_transpose(x3, output_dim, 5, strides=2, padding='same')
        
        out = tf.tanh(logits)
        
        return out

In [81]:
def discriminator(x, reuse=False, alpha=0.2, drop_rate=0., num_classes=10, size_mult=64):
    with tf.variable_scope('discriminator', reuse=reuse):
        x = tf.layers.dropout(x, rate=drop_rate/2.5)
        
        # Input layer is 32x32x3
        x1 = tf.layers.conv2d(x, size_mult, 3, strides=2, padding='same')
        relu1 = tf.maximum(alpha * x1, x1)
        relu1 = tf.layers.dropout(relu1, rate=drop_rate)
        
        x2 = tf.layers.conv2d(relu1, size_mult, 3, strides=2, padding='same')
        bn2 = tf.layers.batch_normalization(x2, training=True)
        relu2 = tf.maximum(alpha * x2, x2)
        
        
        x3 = tf.layers.conv2d(relu2, size_mult, 3, strides=2, padding='same')
        bn3 = tf.layers.batch_normalization(x3, training=True)
        relu3 = tf.maximum(alpha * bn3, bn3)
        relu3 = tf.layers.dropout(relu3, rate=drop_rate)
        
        x4 = tf.layers.conv2d(relu3, 2 * size_mult, 3, strides=1, padding='same')
        bn4 = tf.layers.batch_normalization(x4, training=True)
        relu4 = tf.maximum(alpha * bn4, bn4)
        
        x5 = tf.layers.conv2d(relu4, 2 * size_mult, 3, strides=1, padding='same')
        bn5 = tf.layers.batch_normalization(x5, training=True)
        relu5 = tf.maximum(alpha * bn5, bn5)
        
        x6 = tf.layers.conv2d(relu5, 2 * size_mult, 3, strides=2, padding='same')
        bn6 = tf.layers.batch_normalization(x6, training=True)
        relu6 = tf.maximum(alpha * bn6, bn6)
        relu6 = tf.layers.dropout(relu6, rate=drop_rate)
        
        x7 = tf.layers.conv2d(relu5, 2 * size_mult, 3, strides=1, padding='valid')
        # Don't use bn on this layer, because bn would set the mean of each feature
        # to the bn mu parameter.
        # This layer is used for the feature matching loss, which only works if
        # the means can be different when the discriminator is run on the data than
        # when the discriminator is run on the generator samples.
        relu7 = tf.maximum(alpha * x7, x7)
        
        # Flatten it by global average pooling
        features = tf.reduce_mean(relu7, (1, 2))
        
        # Set class_logits to be the inputs to a softmax distribution over the different classes
        class_logits = tf.layers.dense(features, num_classes + extra_class)
        
        
        # Set gan_logits such that P(input is real | input) = sigmoid(gan_logits).
        # Keep in mind that class_logits gives you the probability distribution over all the real
        # classes and the fake class. You need to work out how to transform this multiclass softmax
        # distribution into a binary real-vs-fake decision that can be described with a sigmoid.
        # Numerical stability is very important.
        # You'll probably need to use this numerical stability trick:
        # log sum_i exp a_i = m + log sum_i exp(a_i - m).
        # This is numerically stable when m = max_i a_i.
        # (It helps to think about what goes wrong when...
        #   1. One value of a_i is very large
        #   2. All the values of a_i are very negative
        # This trick and this value of m fix both those cases, but the naive implementation and
        # other values of m encounter various problems)
        
        if extra_class:
            real_class_logits, fake_class_logits = tf.split(class_logits, [num_classes, 1], 1)
            assert fake_class_logits.get_shape()[1] == 1, fake_class_logits.get_shape()
            fake_class_logits = tf.squeeze(fake_class_logits)
        else:
            real_class_logits = class_logits
            fake_class_logits = 0.
        
        mx = tf.reduce_max(real_class_logits, 1, keep_dims=True)
        stable_real_class_logits = real_class_logits - mx

        gan_logits = tf.log(tf.reduce_sum(tf.exp(stable_real_class_logits), 1)) + tf.squeeze(mx) - fake_class_logits
        
        out = tf.nn.softmax(class_logits)
        
        return out, class_logits, gan_logits, features

In [82]:
def model_loss(input_real, input_z, output_dim, y, num_classes, label_mask, alpha=0.2, drop_rate=0.):
    """
    Get the loss for the discriminator and generator
    :param input_real: Images from the real dataset
    :param input_z: Z input
    :param output_dim: The number of channels in the output image
    :param y: Integer class labels
    :param num_classes: The number of classes
    :param alpha: The slope of the left half of leaky ReLU activation
    :param drop_rate: The probability of dropping a hidden unit
    :return: A tuple of (discriminator loss, generator loss)
    """
    
    
    # These numbers multiply the size of each layer of the generator and the discriminator,
    # respectively. You can reduce them to run your code faster for debugging purposes.
    g_size_mult = 32
    d_size_mult = 64
    
    # Here we run the generator and the discriminator
    g_model = generator(input_z, output_dim, alpha=alpha, size_mult=g_size_mult)
    d_on_data = discriminator(input_real, alpha=alpha, drop_rate=drop_rate, size_mult=d_size_mult)
    d_model_real, class_logits_on_data, gan_logits_on_data, data_features = d_on_data
    d_on_samples = discriminator(g_model, reuse=True, alpha=alpha, drop_rate=drop_rate, size_mult=d_size_mult)
    d_model_fake, class_logits_on_samples, gan_logits_on_samples, sample_features = d_on_samples
    
    
    # Here we compute `d_loss`, the loss for the discriminator.
    # This should combine two different losses:
    #  1. The loss for the GAN problem, where we minimize the cross-entropy for the binary
    #     real-vs-fake classification problem.
    #  2. The loss for the SVHN digit classification problem, where we minimize the cross-entropy
    #     for the multi-class softmax. For this one we use the labels. Don't forget to ignore
    #     use `label_mask` to ignore the examples that we are pretending are unlabeled for the
    #     semi-supervised learning problem.
    d_loss_real = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=gan_logits_on_data,
                                                labels=tf.ones_like(gan_logits_on_data)))
    d_loss_fake = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=gan_logits_on_samples,
                                                labels=tf.zeros_like(gan_logits_on_samples)))
    y = tf.squeeze(y)
    class_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=class_logits_on_data,
                                                                  labels=tf.one_hot(y, num_classes + extra_class,
                                                                                    dtype=tf.float32))
    class_cross_entropy = tf.squeeze(class_cross_entropy)
    label_mask = tf.squeeze(tf.to_float(label_mask))
    d_loss_class = tf.reduce_sum(label_mask * class_cross_entropy) / tf.maximum(1., tf.reduce_sum(label_mask))
    d_loss = d_loss_class + d_loss_real + d_loss_fake
    
    # Here we set `g_loss` to the "feature matching" loss invented by Tim Salimans at OpenAI.
    # This loss consists of minimizing the absolute difference between the expected features
    # on the data and the expected features on the generated samples.
    # This loss works better for semi-supervised learning than the tradition GAN losses.
    data_moments = tf.reduce_mean(data_features, axis=0)
    sample_moments = tf.reduce_mean(sample_features, axis=0)
    g_loss = tf.reduce_mean(tf.abs(data_moments - sample_moments))

    pred_class = tf.cast(tf.argmax(class_logits_on_data, 1), tf.int32)
    eq = tf.equal(tf.squeeze(y), pred_class)
    correct = tf.reduce_sum(tf.to_float(eq))
    masked_correct = tf.reduce_sum(label_mask * tf.to_float(eq))
    
    return d_loss, g_loss, correct, masked_correct, g_model

In [83]:
def model_opt(d_loss, g_loss, learning_rate, beta1):
    """
    Get optimization operations
    :param d_loss: Discriminator loss Tensor
    :param g_loss: Generator loss Tensor
    :param learning_rate: Learning Rate Placeholder
    :param beta1: The exponential decay rate for the 1st moment in the optimizer
    :return: A tuple of (discriminator training operation, generator training operation)
    """
    # Get weights and biases to update. Get them separately for the discriminator and the generator
    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    for t in t_vars:
        assert t in d_vars or t in g_vars

    # Minimize both players' costs simultaneously
    d_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
    g_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(g_loss, var_list=g_vars)
    shrink_lr = tf.assign(learning_rate, learning_rate * 0.9)
    
    return d_train_opt, g_train_opt, shrink_lr

In [84]:
class GAN:
    """
    A GAN model.
    :param real_size: The shape of the real data.
    :param z_size: The number of entries in the z code vector.
    :param learnin_rate: The learning rate to use for Adam.
    :param num_classes: The number of classes to recognize.
    :param alpha: The slope of the left half of the leaky ReLU activation
    :param beta1: The beta1 parameter for Adam.
    """
    def __init__(self, real_size, z_size, learning_rate, num_classes=10, alpha=0.2, beta1=0.5):
        tf.reset_default_graph()
        
        self.learning_rate = tf.Variable(learning_rate, trainable=False)
        self.input_real, self.input_z, self.y, self.label_mask = model_inputs(real_size, z_size)
        self.drop_rate = tf.placeholder_with_default(.5, (), "drop_rate")
        
        loss_results = model_loss(self.input_real, self.input_z,
                                              real_size[2], self.y, num_classes, label_mask=self.label_mask,
                                                                          alpha=0.2,
                                                           drop_rate=self.drop_rate)
        self.d_loss, self.g_loss, self.correct, self.masked_correct, self.samples = loss_results
        
        self.d_opt, self.g_opt, self.shrink_lr = model_opt(self.d_loss, self.g_loss, self.learning_rate, beta1)

In [85]:
def view_samples(epoch, samples, nrows, ncols, figsize=(5,5)):
    fig, axes = plt.subplots(figsize=figsize, nrows=nrows, ncols=ncols, 
                             sharey=True, sharex=True)
    for ax, img in zip(axes.flatten(), samples[epoch]):
        ax.axis('off')
        img = ((img - img.min())*255 / (img.max() - img.min())).astype(np.uint8)
        ax.set_adjustable('box-forced')
        im = ax.imshow(img)
   
    plt.subplots_adjust(wspace=0, hspace=0)
    return fig, axes

In [130]:
def train(net,epochs,dataSet,batch_size, figsize=(5,5)):
    
    saver = tf.train.Saver()
    sample_z = np.random.normal(0, 1, size=(50, z_size))

    samples, train_accuracies, test_accuracies = [], [], []
    steps = 0

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for e in range(epochs):
            print("Epoch",e)
            
            t1e = time.time()
            num_examples = 0
            num_correct = 0
#             for x, y, label_mask in dataset.batches(batch_size):
            
            for x, y, label_mask in dataSet.get_batches():
                
                assert 'int' in str(y.dtype)
                steps += 1
                num_examples += label_mask.sum()

                # Sample random noise for G
                batch_z = np.random.normal(0, 1, size=(batch_size, z_size))

                # Run optimizers
                t1 = time.time()
                _, _, correct = sess.run([net.d_opt, net.g_opt, net.masked_correct],
                                         feed_dict={net.input_real: x, net.input_z: batch_z,
                                                    net.y : y, net.label_mask : label_mask})
                t2 = time.time()
                num_correct += correct
            sess.run([net.shrink_lr])
            
            
            train_accuracy = num_correct / float(num_examples)
            
            print("\t\tClassifier train accuracy: ", train_accuracy)
            
            num_examples = 0
            num_correct = 0
            for x, y in dataSet.get_valid_batches():
                assert 'int' in str(y.dtype)
                num_examples += x.shape[0]

                correct, = sess.run([net.correct], feed_dict={net.input_real: x,
                                                   net.y : y,
                                                   net.drop_rate: 0.})
                num_correct += correct
            
            test_accuracy = num_correct / float(num_examples)
            print("\t\tClassifier test accuracy", test_accuracy)
            print("\t\tStep time: ", t2 - t1)
            t2e = time.time()
            print("\t\tEpoch time: ", t2e - t1e)
            
            
            gen_samples = sess.run(
                                   net.samples,
                                   feed_dict={net.input_z: sample_z})
            samples.append(gen_samples)
#             _ = view_samples(-1, samples, 5, 10, figsize=figsize)
#             plt.show()
            
            
            # Save history of accuracies to view after training
            train_accuracies.append(train_accuracy)
            test_accuracies.append(test_accuracy)
            

        saver.save(sess, './checkpoints/generator.ckpt')

    with open('samples.pkl', 'wb') as f:
        pkl.dump(samples, f)
    
    return train_accuracies, test_accuracies, samples

In [118]:
!mkdir checkpoints

mkdir: cannot create directory ‘checkpoints’: File exists


In [147]:
net = GAN(real_size, z_size, learning_rate)

In [148]:
s = DataSet(128,(224,224),datagen,20000,2000)

Found 19810 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.
Found 2614 images belonging to 10 classes.


In [ ]:
# dataset = Dataset(trainset, testset)
epochs = 77
train_accuracies, test_accuracies, samples = train(net,epochs, s, batch_size, figsize=(10,5))

Epoch 0
		Classifier train accuracy:  0.3210452402447588
		Classifier test accuracy 0.45092518101367657
		Step time:  0.4745016098022461
		Epoch time:  610.9776084423065
Epoch 1
		Classifier train accuracy:  0.6909419199518507
		Classifier test accuracy 0.6154465004022526
		Step time:  0.48914623260498047
		Epoch time:  610.2359771728516
Epoch 2
		Classifier train accuracy:  0.8668873507874411
		Classifier test accuracy 0.6568785197103781
		Step time:  0.4700751304626465
		Epoch time:  609.82488322258
Epoch 3
		Classifier train accuracy:  0.9328418096097904
		Classifier test accuracy 0.7152051488334674
		Step time:  0.477447509765625
		Epoch time:  610.7562038898468
Epoch 4
		Classifier train accuracy:  0.960577791152573
		Classifier test accuracy 0.7228479485116653
		Step time:  0.48784375190734863
		Epoch time:  612.0012636184692
Epoch 5
		Classifier train accuracy:  0.9723643294212058
		Classifier test accuracy 0.7168141592920354
		Step time:  0.47682666778564453
		Epoch time:  611.